# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [19]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [20]:
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [21]:
# 1: Agrupar el dataset por CustomerID.
# Sumar el amount_spent para cada cliente.
# Guardar el resultado en un nuevo DataFrame con columnas: CustomerID y total_spent.

customer_spending = orders.groupby("CustomerID")["amount_spent"].sum().reset_index()
customer_spending.rename(columns={"amount_spent": "total_spent"}, inplace=True)
customer_spending.head()

,CustomerID,total_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [22]:
# Calculate thresholds
p95 = customer_spending["total_spent"].quantile(0.95)
p90 = customer_spending["total_spent"].quantile(0.90)

# Label customers
def label_customer(spent):
    if spent > p90:
        return "VIP"
    else:
        return "Regular"

customer_spending["customer_type"] = customer_spending["total_spent"].apply(label_customer)
customer_spending.head()

,CustomerID,total_spent,customer_type
0,12346,77183.60,VIP
1,12347,4310.00,VIP
2,12348,1797.24,Regular
3,12349,1757.55,Regular
4,12350,334.40,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [23]:
# Merge the labeled customer types back with the original 'orders' dataframe.
# We'll use it to get the country for each customer.
# Drop duplicate CustomerID-Country pairs to avoid counting the same customer multiple times.
# Filter only VIP customers.
# Group by Country and count the number of VIP customers.
# Sort the result to see which country has the most VIPs.

# Merge customer types back to orders
orders_with_labels = orders.merge(customer_spending[["CustomerID", "customer_type"]], on="CustomerID")

# Drop duplicates to count each customer once per country
unique_customers = orders_with_labels[["CustomerID", "Country", "customer_type"]].drop_duplicates()

# Filter VIPs
vip_customers_by_country = unique_customers[unique_customers["customer_type"] == "VIP"]

# Group by country and count
vip_counts = vip_customers_by_country.groupby("Country")["CustomerID"].count().reset_index(name="VIP_count")
vip_counts.sort_values(by="VIP_count", ascending=False).head()

,Country,VIP_count
21,United Kingdom,354
9,Germany,22
8,France,18
18,Spain,7
20,Switzerland,5


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [24]:
# Use the same unique_customers DataFrame from Q2.
# Filter customers whose type is either VIP or Preferred.
# Group by country and count the number of such customers.
# Sort to see which country has the most.

# Filter VIP + Preferred
vip_preferred = unique_customers[unique_customers["customer_type"].isin(["VIP", "Preferred"])]
# .isin()es un "o", es un método de pandas que sirve para filtrar filas donde una columna tiene un valor que esté dentro de una lista de valores.

# Group by country and count
vip_preferred_counts = vip_preferred.groupby("Country")["CustomerID"].count().reset_index(name="VIP_Pref_count")
vip_preferred_counts.sort_values(by="VIP_Pref_count", ascending=False).head()

,Country,VIP_Pref_count
21,United Kingdom,354
9,Germany,22
8,France,18
18,Spain,7
20,Switzerland,5
